In [ ]:
# Run it with Python3

In [10]:
#Append path
import sys
sys.path.append('../dataset')
sys.path.append('..')

# Import OS
import os

# CQRI to get tweets
from QCRI import CQRI

# Librairies for computations and ML
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix

# Functions
import import_ipynb # Remove if you don't use Jupyter
from featureExtraction_functions import tf_idf # Remove if you don't use Jupyter

# Python utility tools 
import re # to remove URL's from tweets
import datetime,time
import string

In [11]:
# Extract events ID
dataset = CQRI('../twitter.txt')
events = dataset.get_dict()   # start Jupyter with the command line: --NotebookApp.iopub_data_rate_limit=10000000000
                              # for ex.: ipython3 notebook --NotebookApp.iopub_data_rate_limit=10000000000
#print(events)

In [57]:
# Training on the whole tweets dataset to learn the vocabulary
nbrEvents = 0
S_list_total = [];
for keyEvent in events:
    if os.path.isfile('events/'+keyEvent+'.json'):  # check that the event file exists
        nbrEvents += 1  
        dico = dataset.get_tweets('events/'+keyEvent+'.json')
        # Extraction of the tweet strings and date
        for keyTweet in dico:           #iterates over the keys
            date,text = dico[keyTweet]
            #print(date)

            if type(text) == str and type(date) == datetime.datetime:  # because sometimes text is just nothing, so of type None
                text = re.sub(r"http\S+", "", text) # remove URL's
                text = re.sub(r"@\S+","",text)     # Optional: remove user names
                #text=re.sub(r"\\xa0\S+","",text)
                S_list_total.append(text)
            
            
vectorizer = TfidfVectorizer()
dummy = vectorizer.fit(S_list_total)
print(vectorizer.vocabulary_)

{'filled': 29544, 'yjuw6xykvp': 85438, 'claudio': 18597, 'skyrocket': 69076, 'hj98mosnpc': 36278, 'g9d5fpujq6': 31812, '40oezd6aox': 3773, 'hcszfgeuo4': 35419, 'posner': 59693, 'khe': 43423, 'valde': 78493, 'belvedere': 13566, 'fidelcastropic': 29472, 'weaponizing': 81105, 'vakhckuhxrpic': 78487, 'iapoliticspic': 37957, 'dominio': 24097, 'techeblog': 72994, 'coppic': 20038, '0dmtd6k9rl': 225, 'dbvf77vn3j': 22006, 'vsb': 79987, 'brenda': 15435, 'kamloops': 42886, 'i2bru4gh74': 37744, 'comp': 19330, 'duosb37ie4': 24889, 'reparations': 64335, 'whitespic': 81702, 'lmjeggbejf': 46623, '2wjoq0yghl': 2800, 'interrested': 39538, 'oby535ktxy': 54993, 'birth': 14176, 'dkeqqnnuhx': 23750, 'lseinceqs2': 47302, 'trial': 75235, 'glcdfmwovn': 33010, '70rku5ixzq': 6052, 'frga6dvwba': 30884, 'zlhcmwqnwq': 86893, 'yannick': 84922, 'kkil2pnfnu': 43748, 'xeon44gjjf': 83766, 'carli': 17018, 'simmons': 68788, 'resurface': 64619, 'terms': 73249, 'saoymuyplo': 66945, 'foreman': 30403, 'men': 49266, 'bpq41tjdf

In [109]:
# Parameters
N=12 #reference number of intervals
K = 5000

featuresTensor = []; # list containing tuples (matrixOfFeatures,label), where matrixOfFeatures is a matrix of size K x (number of time interval)

for keyEvent in events:
    if os.path.isfile('events/'+keyEvent+'.json'):  # check that the event file exists
        dico = dataset.get_tweets('events/'+keyEvent+'.json')
        ev = events[keyEvent]
        label = ev[1]

        S_list = [];
        date_list = [];

        # Extraction of the tweet strings and date
        for keyTweet in dico:           #iterates over the keys
            date,text = dico[keyTweet]
            #print(date)

            if type(text) == str and type(date) == datetime.datetime:  # because sometimes text is just nothing, so of type None
                text = re.sub(r"http\S+", "", text) # remove URL's
                text = re.sub(r"@\S+","",text)     # Optional: remove user names
                #text=re.sub(r"\\xa0\S+","",text)
                if text != '':
                    S_list.append(text)
                    date = time.mktime(date.timetuple()) # number of seconds since 1 January 1970
                    date_list.append(date)

        # Sorting w.r.t the date values 
        idx = np.argsort(date_list)
        date_list=np.array(date_list)
        date_list=date_list[idx] # final list is a numpy array sorted in ascending order
        S_list_sorted=[0]*len(S_list)
        for i in range(0,len(S_list)):
            S_list_sorted[i]=S_list[idx[i]]  # S_list is sorted in ascending order w.r.t the date values

        # Time interval
        timeStart = date_list[0]
        timeEnd = date_list[-1]
        totalTimeInterval = timeEnd - timeStart
        timeStep = totalTimeInterval / N  # in [seconds], timeStep is the lowercase 'l' in the paper
        timeStep_init = timeStep # save it for the cut of the intervals

        if timeStep > totalTimeInterval:
            print("timeStep > totalTimeInterval")

        #print(S_list_sorted)

        # Dividing in intervals of duration equal to timeStep, and finding the continuous super-interval, i.e. the longest serie of intervals without empty space
        count_save = 0
        while(True):
            # Dividing in intervals
            S_list_intervals = []  # list of sublists containing tweets belonging to the same interval (each sublist is one interval)
            n = 0 # global counter for the time steps
            i = 0 # counter within an interval
            timeUp = 0
            while timeUp < timeStart + totalTimeInterval:
                timeDown = timeStart + n * timeStep
                timeUp = timeDown + timeStep

                interval = []
                while i < len(date_list):
                    if date_list[i] <= timeUp and date_list[i] >= timeDown:
                        interval.append(S_list_sorted[i])
                        i += 1
                    else:
                        break
                S_list_intervals.append(interval)
                n += 1


            #S_list_intervalsR = [x for x in S_list_intervals if x!=[]]

            #print(S_list_intervals)

            #max_interval = get_max_interval(S_list_intervals)


            # Contiuous interval computation
            continuous_intervals=[] # list of sublists of subsublists: 
                                    # each sublist is a continuous super-interval (continuous = no empty space between intervals)
                                    # each subsublist is one interval inside the continuous super-interval
            count_list=[] # list of the length of the continuous super-intervals
            temp=[]
            count=0
            for elem in S_list_intervals: 
                if(elem==[]):
                    if(temp!=[]):
                        continuous_intervals.append(temp)
                        count_list.append(count)
                    temp=[]
                    count=0
                else:
                    temp.append(elem)
                    count+=1
                    if(elem==S_list_intervals[-1]):
                        continuous_intervals.append(temp)
                        count_list.append(count)

            count_list=np.array(count_list)
            idx_max = np.argmax(count_list)
            max_interval = continuous_intervals[idx_max]  # super-interval covering the longest time span
            count_max = count_list[idx_max]               # number of intervals (and so time steps) in max_interval 
            print("Count_max = ",count_max)

            if (count_max < N and count_max > count_save): # Half the time step and restart at the beginning of while(True)
                #print('Redo')
                timeStep = timeStep/2 # shorten the time interval by doubling N
                count_save = count_max
                max_interval_save = max_interval 
            else:                                         # Output max_interval and count_max
                #print('Done')
                if timeStep != timeStep_init:
                    max_interval = max_interval_save # when outputting take the previous iteration result, that was the best because current iteration didn't improve
                    print("Final count = ", count_save)
                    print("Final interval content:\n")
                    #print(max_interval_save)
                else:
                    print("Final count = ", count_max)
                    print("Final interval content:\n")
                    #print(max_interval)

                break

        # Check lengths
        countLen_maxInterval = 0
        countLen_S_list = 0;
        for elem in max_interval:
            countLen_maxInterval += len(elem)
        for elem2 in S_list_intervals:
            if elem2 != []:
                countLen_S_list += len(elem2)
        if countLen_maxInterval > countLen_S_list:
            print("Problem: there shouldn't be more tweets in the biggest interval than the total number of tweets") 
            
        # Apply tf_idf on each interval of the super-interval
        featuresMat = np.zeros((K,len(max_interval)))
        #print(max_interval)
        for ii in range(0,len(max_interval)):
            #featuresMat[:,ii] = tf_idf(max_interval[ii],True,K)   # to modify to take each interval separately
            tmp = vectorizer.transform(max_interval[ii])
            array = tmp.toarray()
            #print(array.shape)
            vec = np.reshape(array,(array.shape[0]*array.shape[1],1))
            vec = np.ndarray.flatten(vec)
            vec[::-1].sort()
            vec = vec[0:K]
            #print(vec)
            #print(vec.shape) 
            #vec = array[np.nonzero(array)]
            #vec[::-1].sort()
            #vec = np.transpose(vec[0:K])
            featuresMat[:,ii] = vec
        #print(featuresMat)

        featuresTensor.append((featuresMat,label))

        #break # TO TEST ONLY ONE EVENT
#print(featuresTensor)

Count_max =  5
Count_max =  5
Final count =  5
Final interval content:

Count_max =  4
Count_max =  8
Count_max =  6
Final count =  8
Final interval content:

Count_max =  12
Final count =  12
Final interval content:

Count_max =  2
Count_max =  1
Final count =  2
Final interval content:

Count_max =  4
Count_max =  5
Count_max =  4
Final count =  5
Final interval content:

Count_max =  12
Final count =  12
Final interval content:

Count_max =  5
Count_max =  5
Final count =  5
Final interval content:

Count_max =  1
Count_max =  1
Final count =  1
Final interval content:

Count_max =  1
Count_max =  1
Final count =  1
Final interval content:

Count_max =  7
Count_max =  3
Final count =  7
Final interval content:

Count_max =  3
Count_max =  2
Final count =  3
Final interval content:

Count_max =  9
Count_max =  3
Final count =  9
Final interval content:

Count_max =  3
Count_max =  5
Count_max =  5
Final count =  5
Final interval content:

Count_max =  3
Count_max =  1
Final count =  

KeyboardInterrupt: 

In [2]:
#features = tf_idf(S_list,True)   # to modify to take each interval separately
#print(features)